# **Uplift-моделирование**



1.   скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention
2.   там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.
3.   сделать разбиение набора данных не тренировочную и тестовую выборки
4.   сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)
5.   провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями
6.   в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей
7.   построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево
8.   **(опционально)** для модели S learner (модель с дополнительным признаком коммуникации) построить зависимость таргета (конверсии - поле conversion) от значения uplift: 1) сделать прогноз и получить uplift для тестовой выборки 2) отсортировать тестовую выборку по uplift по убыванию 3) разбить на децили (pandas qcut вам в помощь) 4) для каждого дециля посчитать среднюю conversion
9.  **(опционально)** построить модель UpliftRandomForestClassifier и попытаться описать словами полученное дерево



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install scikit-uplift

In [6]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 23 kB/s 


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel
from sklift.models import ClassTransformation
from sklift.models import TwoModels

### **Подготовка данных**

Загрузка данных

In [56]:
df = pd.read_csv('/content/drive/MyDrive/data.csv')
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0


Переименование полей conversion и offer

In [57]:
df.rename(columns={'conversion': 'target', 'offer': 'treatment'}, inplace=True)
df.columns

Index(['recency', 'history', 'used_discount', 'used_bogo', 'zip_code',
       'is_referral', 'channel', 'treatment', 'target'],
      dtype='object')

Привидение поля treatment к бинарному виду

In [58]:
df['treatment_temp'] = 1
df.loc[df.treatment == 'No Offer', 'treatment_temp'] = 0


In [59]:
df.drop(columns='treatment', inplace=True)
df.rename(columns={'treatment_temp': 'treatment'}, inplace=True)
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,target,treatment
0,10,142.44,1,0,Surburban,0,Phone,0,1
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,0,1


Проверка типов данных полей

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recency        64000 non-null  int64  
 1   history        64000 non-null  float64
 2   used_discount  64000 non-null  int64  
 3   used_bogo      64000 non-null  int64  
 4   zip_code       64000 non-null  object 
 5   is_referral    64000 non-null  int64  
 6   channel        64000 non-null  object 
 7   target         64000 non-null  int64  
 8   treatment      64000 non-null  int64  
dtypes: float64(1), int64(6), object(2)
memory usage: 4.4+ MB


Выделение категориальных признаков

In [61]:
cat_features = ['zip_code', 'channel']

Разбиение набора данных на тренировочную и тестовую выборку

In [62]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], random_state=123, test_size=0.3)

In [63]:
treatment_train = X_train.treatment
X_train.drop(columns='treatment', inplace=True)
treatment_test = X_test.treatment
X_test.drop(columns='treatment', inplace=True)

### **Одна модель с признаком коммуникации (S learner)**

In [95]:
models_results = {
    'model': [],
    'uplift@10%': [],
    'uplift@20%': []
}

In [96]:
models_results['model'].append('SoloModel')

In [97]:
sm = SoloModel(CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True))
sm = sm.fit(X_train, y_train, treatment_train, estimator_fit_params={'cat_features': cat_features})

In [98]:
uplift_sm = sm.predict(X_test)
sm_score = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treatment_test, strategy='by_group', k=0.1)
models_results['uplift@10%'].append(sm_score)

In [99]:
uplift_sm = sm.predict(X_test)
sm_score = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treatment_test, strategy='by_group', k=0.2)
models_results['uplift@20%'].append(sm_score)

Результаты работы SoloModel

In [101]:
models_results

{'model': ['SoloModel'],
 'uplift@10%': [0.08925430023455824],
 'uplift@20%': [0.0789944802657288]}

### **Модель с трансформацией таргета**

In [102]:
models_results['model'].append('ClassTransformation')

In [103]:
ct = ClassTransformation(CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True))
ct = ct.fit(X_train, y_train, treatment_train, estimator_fit_params={'cat_features': cat_features})

In [104]:
uplift_ct = ct.predict(X_test)
ct_score = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treatment_test, strategy='by_group', k=0.1)
models_results['uplift@10%'].append(ct_score)

In [105]:
uplift_ct = ct.predict(X_test)
ct_score = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treatment_test, strategy='by_group', k=0.2)
models_results['uplift@20%'].append(ct_score)

Результаты работы модели ClassTransformation

In [106]:
models_results

{'model': ['SoloModel', 'ClassTransformation'],
 'uplift@10%': [0.08925430023455824, 0.1174110633307271],
 'uplift@20%': [0.0789944802657288, 0.09579975087924972]}

### **Две независимые модели**

In [107]:
models_results['model'].append('TwoModels')

In [108]:
tm = TwoModels(
    estimator_trmnt=CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True),
    estimator_ctrl=CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True),
    method='vanilla'
)

In [109]:
tm = tm.fit(
    X_train, y_train, treatment_train,
    estimator_trmnt_fit_params={'cat_features': cat_features},
    estimator_ctrl_fit_params={'cat_features': cat_features}
)

In [110]:
uplift_tm = tm.predict(X_test)
tm_score = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treatment_test, strategy='by_group', k=0.1)
models_results['uplift@10%'].append(tm_score)

In [111]:
uplift_tm = tm.predict(X_test)
tm_score = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treatment_test, strategy='by_group', k=0.2)
models_results['uplift@20%'].append(tm_score)

Результаты работы модели TwoModels

In [112]:
models_results

{'model': ['SoloModel', 'ClassTransformation', 'TwoModels'],
 'uplift@10%': [0.08925430023455824, 0.1174110633307271, 0.07830580531665363],
 'uplift@20%': [0.0789944802657288, 0.09579975087924972, 0.06843890435717076]}

### **Результаты**

In [114]:
results = pd.DataFrame(data=models_results)
results

,model,uplift@10%,uplift@20%
0,SoloModel,0.089254,0.078994
1,ClassTransformation,0.117411,0.095800
2,TwoModels,0.078306,0.068439
